In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 데이터셋 로드
food = pd.read_csv('./foodData4_24.csv')
user_data = pd.read_csv('./123456.csv')    # 유저 데이터셋 예시

# 결측값 처리(Keywords열의 데이터 중 nan값 공백으로 처리)
# 현재 food 데이터에 일부 깨진데이터(?)가 있어서 결측값 처리해야 정상작동
food['Keywords'] = food['Keywords'].fillna('')

# TF-IDF(Term Frequency-Inverse Document Frequency)
# 텍스트 데이터의 통계적인 가중치를 계산, 처리해주는 기능(자연어 처리)

# TfidfVectorizer : 문서를 벡터 표현으로 바꿔주는 기능
# stop_words='english' => 영어의 일반적인 불용어 삭제(and, is, the, this 등 단어와 상관없는 문자)
tfidf = TfidfVectorizer(stop_words='english')

# food의 Keywords를 단어로 구별
# 단어로 구별된 Keywords를 TF-IDF 행렬로 생성
tfidf_matrix = tfidf.fit_transform(food['Keywords'])

# 사용자 정보를 담을 빈 딕셔너리 생성
users = {}

# 데이터프레임 순회
for index, row in user_data.iterrows():
    # 각 유저의 정보 추출
    user_name = row['이름']
    user_height = row['키']
    user_weight = row['몸무게']
    user_gender = row['성별']
    user_keywords = row['키워드']

    # BMI 계산
    height_m = user_height / 100
    weight_kg = user_weight
    bmi = weight_kg / (height_m ** 2)

    # 추정된 칼로리 요구량 계산
    if user_gender == '남':
        bmr = 88.362 + (13.397 * weight_kg) + (4.799 * height_m * 100) - (5.677 * 33)
    else:
        bmr = 447.593 + (9.247 * weight_kg) + (3.098 * height_m * 100) - (4.330 * 33)

    calories_threshold = bmr * 1.2

    # 사용자 정보를 딕셔너리에 추가
    # 사용자의 TF-IDF 행렬을 구성합니다.
    user_tfidf_matrix = tfidf.transform([user_keywords])
    users[user_name] = {'Keywords': user_keywords, 'Calories_threshold': calories_threshold, 'TFIDF_matrix': user_tfidf_matrix}

# 추천 함수 정의
def get_recommendations_break(user_keywords, user_tfidf_matrix, tfidf_matrix, food_data, calories_threshold):
    # 사용자 키워드 행렬과 food 데이터의 키워드 행렬의 코사인 유사도 계산
    cosine_sim = cosine_similarity(user_tfidf_matrix, tfidf_matrix)
   
    # 코사인 유사도가 계산되었으므로, 유사도가 높은 상위 음식을 추천 리스트에 추가합니다.
    # 유사도를 기준으로 내림차순으로 정렬하여 상위 음식을 추천 리스트에 추가합니다.
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 상위 음식을 추천 리스트에 추가합니다.
    recommended_food_break = []
   
    for index, score in sim_scores:
        food_name = food_data.iloc[index]['Name']
        food_calories = food_data.iloc[index]['Calories']
        if food_calories <= calories_threshold * 0.1:
            recommended_food_break.append({'Name': food_name, 'Calories': food_calories}) 
    
    return recommended_food_break

def get_recommendations_lunch(user_keywords, user_tfidf_matrix, tfidf_matrix, food_data, calories_threshold):
    # 사용자 키워드 행렬과 food 데이터의 키워드 행렬의 코사인 유사도 계산
    cosine_sim = cosine_similarity(user_tfidf_matrix, tfidf_matrix)
   
    # 코사인 유사도가 계산되었으므로, 유사도가 높은 상위 음식을 추천 리스트에 추가합니다.
    # 유사도를 기준으로 내림차순으로 정렬하여 상위 음식을 추천 리스트에 추가합니다.
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 상위 음식을 추천 리스트에 추가합니다.
    recommended_food_lunch = []
   
    for index, score in sim_scores:
        food_name = food_data.iloc[index]['Name']
        food_calories = food_data.iloc[index]['Calories']
        if food_calories <= calories_threshold * 0.45:
            recommended_food_lunch.append({'Name': food_name, 'Calories': food_calories}) 
    
    return recommended_food_lunch

def get_recommendations_dinner(user_keywords, user_tfidf_matrix, tfidf_matrix, food_data, calories_threshold):
    # 사용자 키워드 행렬과 food 데이터의 키워드 행렬의 코사인 유사도 계산
    cosine_sim = cosine_similarity(user_tfidf_matrix, tfidf_matrix)
   
    # 코사인 유사도가 계산되었으므로, 유사도가 높은 상위 음식을 추천 리스트에 추가합니다.
    # 유사도를 기준으로 내림차순으로 정렬하여 상위 음식을 추천 리스트에 추가합니다.
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 상위 음식을 추천 리스트에 추가합니다.
    recommended_food_dinner = []
   
    for index, score in sim_scores:
        food_name = food_data.iloc[index]['Name']
        food_calories = food_data.iloc[index]['Calories']
        if food_calories <= calories_threshold * 0.15:
            recommended_food_dinner.append({'Name': food_name, 'Calories': food_calories}) 
    
    return recommended_food_dinner

# 결과 출력
for user_name, user_info in users.items():
    print(user_name, "님의 필터된 음식 추천:")
    print("아침메뉴 추천:", user_info['Calories_threshold']*0.1, "kcal 이내")
    recommendations_break = get_recommendations_break(user_info['Keywords'], user_info['TFIDF_matrix'], tfidf_matrix, food, user_info['Calories_threshold'])
    print(pd.DataFrame(recommendations_break[1:100000:1000]))

    print("점심메뉴 추천:", user_info['Calories_threshold']*0.45, "kcal 이내")
    recommendations_lunch = get_recommendations_lunch(user_info['Keywords'], user_info['TFIDF_matrix'], tfidf_matrix, food, user_info['Calories_threshold'])
    print(pd.DataFrame(recommendations_lunch[2:100000:1000]))

    print("저녁메뉴 추천:", user_info['Calories_threshold']*0.15, "kcal 이내")
    recommendations_dinner = get_recommendations_dinner(user_info['Keywords'], user_info['TFIDF_matrix'], tfidf_matrix, food, user_info['Calories_threshold'])
    print(pd.DataFrame(recommendations_dinner[3:100000:1000]))

홍길동 님의 필터된 음식 추천:
아침메뉴 추천: 209.17216800000003 kcal 이내
                                                 Name  Calories
0                       Bittersweet Chocolate Granita     178.3
1           Extract the Most Juice from Citrus Fruits      98.5
2                           Portobello Mushroom Toast      89.9
3                       Grilled Corn With Mint Butter     153.9
4                               Italian stuffed bread       0.0
5                          Lemon &amp; Coriander Soup      34.2
6                                 Vegan Lemon Cookies     190.0
7               Chocolate Blocks With Fruits and Nuts      66.4
8                                  Caramelized Onions     107.6
9                        Golden Autumn Soup (Low Fat)      60.6
10           Carrot &amp; Zucchini Ribbons With Pesto      45.2
11                                  Strawberry Squash     166.3
12                                    Jalapeno Butter     106.6
13                          Stiff Porridge (Oshifi

KeyboardInterrupt: 